In [3]:
## betting loop 
from pydantic import BaseModel
from typing import Literal

# NEED TO KEEP TRACK OF THOSE WHO FOLD


class Player(BaseModel):
    pid : int
    funds : int = 100
    current_hand_betting_status : Literal["active", "fold", "all-in", "inactive"] = "inactive"
    def make_bet(self, pot_state) -> (str, int):
        print(f"Player{self.pid}, given {pot_state}, your choices are:")
        print("  1)fold")
        print(f"  2)call for {pot_state['call_amount']}")
        print(f"  3)raise for at least {pot_state['call_amount']*2}")
        while True:
            action : Literal["call", "fold", "raise"] = input("Enter action fold, call, or raise:")
            if action in ["call", "fold", "raise"]:
                break
        amount = 0
        if action == "raise":
            amount = input("Enter amount")
            pot_state['call_amount'] = int(amount)
        return (action, amount)

    def set_status(self, new_status : Literal["active", "fold", "all-in", "inactive"]):
        self.current_hand_betting_status = new_status

    def hand_status(self):
        return self.current_hand_betting_status




PLAYERS  = [Player(pid = i, current_hand_betting_status = "active") for i in range(0,3)]


class Pot(BaseModel):
    pot_state :int = {"size" : 0, "call_amount": 10}

    def betting_round(self):
        '''pass the small blind index and amount here'''
        ## Assume players has only active players ... how do we keep track if they fold, or go al in? a special, iterable data struture
        players_to_call = len(PLAYERS)
        active_players = len(PLAYERS)
        while players_to_call != 0:
            for player in PLAYERS:
                if players_to_call == 0:
                    break
                if player.hand_status() == "active":
                    print(f"\nplayers_to_call = {players_to_call}")
                    print(f"active_players = {active_players}")
    
                    player_action, amount = player.make_bet(self.pot_state)
                    if player_action == "raise":
                        players_to_call = active_players-1 # all active players
                    if player_action == "fold":
                        players_to_call-=1
                        active_players-=1
                        player.set_status("fold")
                    if player_action == "call":
                        players_to_call -=1
            
        print("betting round over!")
                
            





    



In [ ]:
Pot().betting_round()



players_to_call = 3
active_players = 3
Player0, given {'size': 0, 'call_amount': 10}, your choices are:
  1)fold
  2)call for 10
  3)raise for at least 20


Enter action fold, call, or raise: fold



players_to_call = 2
active_players = 2
Player1, given {'size': 0, 'call_amount': 10}, your choices are:
  1)fold
  2)call for 10
  3)raise for at least 20


Enter action fold, call, or raise: call



players_to_call = 1
active_players = 2
Player2, given {'size': 0, 'call_amount': 10}, your choices are:
  1)fold
  2)call for 10
  3)raise for at least 20


Enter action fold, call, or raise: raise
Enter amount 50



players_to_call = 1
active_players = 2
Player1, given {'size': 0, 'call_amount': 50}, your choices are:
  1)fold
  2)call for 50
  3)raise for at least 100


Enter action fold, call, or raise: raise 200
Enter action fold, call, or raise: call



players_to_call = 0
active_players = 2
Player2, given {'size': 0, 'call_amount': 50}, your choices are:
  1)fold
  2)call for 50
  3)raise for at least 100


In [8]:
from typing_extensions import TypedDict
# import pydantic 

class PotState:
    def __init__(self, call_amount : int):
        self.call_amount : int = call_amount
        self.check_allowed : bool = False
        self.minimum_raise : int = call_amount*2
        self.pot_size : int = 0


In [13]:
from enum import IntEnum
class BoardStage(IntEnum):
    ZERO = 0
    PREFLOP = 1
    FLOP = 2
    TURN = 3
    RIVER = 4

bs = BoardStage.ZERO
bs.name

'ZERO'